In [1]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.14.1-py3-none-any.whl size=267559 sha256=2792a78c48f73cf4648e3cc9b90091373f5cac7bd176894adaf7908eef12717b
  Stored in directory: /root/.cache/pip/wheels/13/01/ff/de2142e172dafcd475f118499db2140c66562e42fd500c460a
Successfully built neo4j


In [2]:
# fetch all required ASN from the cloud
# to fetch all the attributes of each AS online takes quite a lot of time

from neo4j import GraphDatabase

# Neo4j Aura DB credentials
uri = "neo4j+s://28e5b287.databases.neo4j.io"
username = "neo4j"
password = "z9J3DPCCxGYELn99XDdFbFIBnWOwR5fdn4MiG_Nvdck"

# Function to fetch all IDs
def fetch_all_ids(driver):
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN n.id AS id")
        return ["AS" + record["id"] for record in result]

# Connect to Neo4j Aura DB
driver = GraphDatabase.driver(uri, auth=(username, password))

# Fetch and print all IDs
try:
    ids = fetch_all_ids(driver)
finally:
    driver.close()

['AS42708', 'AS50304', 'AS42708', 'AS50304', 'AS3356', 'AS45896', 'AS3356', 'AS45896', 'AS6830', 'AS8758', 'AS6830', 'AS8758', 'AS6939', 'AS6939', 'AS4826', 'AS4826', 'AS38803', 'AS38803', 'AS56203', 'AS56203', 'AS2497', 'AS4777', 'AS2497', 'AS4777', 'AS4844', 'AS37989', 'AS4844', 'AS37989', 'AS4608', 'AS4608', 'AS57821', 'AS57821', 'AS3549', 'AS3549', 'AS1299', 'AS1299', 'AS29608', 'AS29608', 'AS22652', 'AS22652', 'AS57381', 'AS57381', 'AS50300', 'AS50300', 'AS3257', 'AS3257', 'AS7018', 'AS7018', 'AS1836', 'AS1836', 'AS3333', 'AS3333', 'AS2516', 'AS2516', 'AS4637', 'AS4637', 'AS1221', 'AS1221', 'AS10026', 'AS10026', 'AS2519', 'AS2519', 'AS7670', 'AS7670', 'AS18144', 'AS18144', 'AS209', 'AS209', 'AS1273', 'AS1273', 'AS38040', 'AS38040', 'AS9737', 'AS9737', 'AS6762', 'AS6762', 'AS12586', 'AS12586', 'AS2914', 'AS2914', 'AS8220', 'AS8220', 'AS4713', 'AS4713', 'AS174', 'AS174', 'AS1103', 'AS1103', 'AS4651', 'AS4651', 'AS23969', 'AS23969', 'AS4766', 'AS4766', 'AS31025', 'AS31025', 'AS1136',

In [3]:
# remove the duplicaed ids. There are 2 labels in the graph datbase, which contain duplicated ids
ids = list(set(ids))

In [4]:
# Parse ASN report from url
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL from which to scrape content
url = 'https://bgp.potaroo.net/cidr/autnums.html'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the <pre> tag
    pre_tag = soup.find('pre')
    if pre_tag:
        # Find all <a> tags within the <pre> tag
        a_tags = pre_tag.find_all('a')

        # List to store each row of data
        data = []

        # Extract the href attribute and text from each <a> tag
        for a_tag in a_tags:
            href = a_tag.get('href')
            text = a_tag.get_text(strip=True)
            sibling_text = a_tag.next_sibling

            # Append as a tuple to the data list
            data.append((href, text, sibling_text))

        # Create a DataFrame
        df_url = pd.DataFrame(data, columns=['Link', 'Text', 'Sibling Text'])[1:]

        # Optionally, save the DataFrame to a file, e.g., CSV
        # df.to_csv('output.csv', index=False)

    else:
        print("<pre> tag not found in the HTML.")

else:
    print("Failed to retrieve the webpage")


In [5]:
# Check the raw parsing data
df_filter = df_url[df_url['Text'].isin(ids)]
df_filter.head()

In [7]:
import re
import pandas as pd

def clean_as_number(as_number):
    return re.sub(r'\D', '', as_number) if as_number else None

base_url = 'https://bgp.potaroo.net'

dic = {}
for url in df_filter['Link']:
    cur_url = base_url + url

    response = requests.get(cur_url)

    # Check if the request was successful
    if response.status_code == 200:
    # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        ul_tags = soup.find_all('ul')
        text = ""
        for ul in ul_tags:
          text += ul.get_text()
    # Regular expressions to extract the data
    as_number_pattern = r"ASNumber:\s+(\d+)|aut-num:\s+AS(\d+)"
    as_name_pattern = r"ASName:\s+(.+)|as-name:\s+(.+)"
    org_name_pattern = r"OrgName:\s+(.+)|org-name:\s+(.+)"
    country_pattern = r"(?i)Country:\s+(.+)"
    city_pattern = r"City:\s+(.+)"
    state_pattern = r"StateProv:\s+(.+)|State:\s+(.+)"
    adjacent_asn_pattern = r"Upstream Adjacent AS list\n(.+)"
    upstream_pattern = r"Upstream:\s+(\d+)"
    downstream_pattern = r"Downstream:\s+(\d+)"
    rank_pattern = r"Rank\s+AS\s+Type\s+Originate Addr Space\s+\(pfx\)\s+Transit Addr space\s+\(pfx\)\s+Description\n(\d+)"
    if re.search(as_number_pattern, text):
        # Extracting data using regex with checks
        as_number = re.search(as_number_pattern, text).group()
        as_number = clean_as_number(as_number)
        as_name = re.search(as_name_pattern, text).group(1) if re.search(as_name_pattern, text).group(1) else re.search(as_name_pattern, text).group(2)
        org_name_search = re.search(org_name_pattern, text)
        if org_name_search:
            org_name = org_name_search.group(1) if org_name_search.group(1) else org_name_search.group(2)
        else:
            org_name = None
        #org_name = re.search(org_name_pattern, text).group(1) if re.search(org_name_pattern, text).group(1) else re.search(org_name_pattern, text).group(2)
        country = re.search(country_pattern, text).group(1) if re.search(country_pattern, text) else None
        city = re.search(city_pattern, text).group(1) if re.search(city_pattern, text) else None
        state = re.search(state_pattern, text).group(1) if re.search(state_pattern, text) else None
        upstream = re.search(upstream_pattern, text).group(1) if re.search(upstream_pattern, text) else None
        downstream = re.search(downstream_pattern, text).group(1) if re.search(downstream_pattern, text) else None
        rank = re.search(rank_pattern, text).group(1) if re.search(rank_pattern, text) else None

        dic[as_number] = [as_name,org_name, country, city,state,upstream,downstream, rank]

data_tuples = [(key, *values) for key, values in dic.items()]
df = pd.DataFrame(data_tuples,columns=["ASN","as_name","org_name", "country", "city","state", "upstream","downstream", "rank"] )

In [9]:
# Check missing values
df.isna().sum()

ASN            0
as_name        0
org_name      15
country        7
city          41
state         41
upstream       3
downstream     3
rank           3
dtype: int64

In [10]:
# Save it into csv file
df.to_csv('data/attri.csv', index = False)

In [ ]:
# Update the Neo4j Aura database
def update_node(driver, attributes):
    with driver.session() as session:
        session.run(
            """
            MATCH (n) WHERE n.id = $ASN
            SET n.as_name = $as_name, n.org_name = $org_name, n.country = $country,
                n.city = $city, n.state = $state,n.upstream = $upstream,
                n.downstream = $downstream, n.rank = $rank
            """,
            **attributes
        )

try:
    for _, row in df.iterrows():
        update_node(driver, row.to_dict())
finally:
    driver.close()


<ipython-input-31-025aafc83725>:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
